<a href="https://colab.research.google.com/github/Nagi0/bh-traffic-data-mining/blob/main/Fase2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Autores: Bruno Leal Fonseca & Guilherme Namen Pimenta

Gemini 1.5 Pro OO2 – Temperatura 0

Prompt:

Necessito de ajuda para fazer o trabalho abaixo, me informe como proceder em todas as etapas desde a coleta, processamento e geração do relatório final.

O documento descreve um projeto de mineração de dados para analisar padrões de tráfego em Belo Horizonte usando dados de radares. Aqui está um guia passo a passo para realizar este projeto, expandindo os pontos do documento e adicionando detalhes práticos:

Fase 1: Coleta e Tratamento de Dados



1.   Download dos Dados: Baixe os arquivos JSON dos radares do portal de dados abertos da Prefeitura de Belo Horizonte (https://dados.pbh.gov.br/dataset/contagens-volumetricas-de-radares).
2.   Organize os arquivos por mês e dia, como descrito no documento. Considere baixar dados de um período menor inicialmente para testes e desenvolvimento.
3.   Escolha da Via: Selecione uma via principal de Belo Horizonte para focar a análise. Essa escolha reduz a complexidade inicial e permite resultados mais específicos.

4.   Conversão para Formato Tabular: Os dados JSON precisam ser convertidos para um formato tabular (como CSV ou um banco de dados relacional) para facilitar o processamento. Use uma biblioteca Python como pandas para ler os arquivos JSON e criar um DataFrame. Em seguida, exporte o DataFrame para um arquivo CSV ou carregue-o em um banco de dados (PostgreSQL, MySQL, SQLite, etc.).

In [ ]:
import pandas as pd
import glob
import json

# Lista todos os arquivos JSON em um diretório
json_files = glob.glob('ABRIL_2021/20210401/*.json')

# Cria uma lista vazia para armazenar os dados
data = []

# Itera sobre os arquivos JSON
for file in json_files:
    with open(file, 'r') as f:
        for line in f:
            try:
                data.append(json.loads(line))
            except json.JSONDecodeError:
                print(f"Erro ao decodificar JSON em {file}")

# Cria um DataFrame pandas
df = pd.DataFrame(data)
# Seleciona as colunas relevantes
df = df[['ID_EQP', 'DATA_HORA', 'MILESEGUNDO', 'FAIXA', 'ID_DE_ENDEREÇO', 'VELOCIDADE_DA_VIA', 'VELOCIDADE_AFERIDA', 'CLASSIFICACAO', 'TAMANHO', 'NUM_SERIE', 'LATITUDE', 'LONGITUDE', 'ENDEREÇO', 'SENTIDO']]

# Salva o DataFrame em um arquivo CSV
df.to_csv('dados_radares.csv', index=False)


**Crítca**

A base de dados é bem complexa e esta divida em várias arquivos zip contendo uma grande quantidade de informações, foi necessário criar um projeto a parte para reunir as informações necessárias e filtrar os dados. Mesmo filtrando por uma via, a quantidade de informações é muito grande então decidimos filtrar pelo horário de pico da manhã das 07:00 às 09:00.

Uma vantagem foi a criação do código para converter de json para dataframe.

**LLM**
Limpeza e Pré-processamento:

1. Valores Ausentes: Trate valores ausentes (NaN) na tabela. Dependendo da quantidade, você pode remover as linhas com valores ausentes ou imputar valores (média, mediana, etc.).

2. Formatação de Data/Hora: Converta a coluna DATA_HORA para o tipo datetime do pandas para facilitar a análise temporal.

3. Filtragem por Via: Filtre o DataFrame para incluir apenas os dados da via selecionada.

**Crítica**

Para bases de dados simples o procedimento funciona bem. O problema é que a base de dados em questão é muito grande cerca de 12GB compactados. Sendo assim o colab não suporta o procedimento. É necessário um pequeno projeto para gerar o csv para ser processado. O geração da base é um processo bem complexo e demanda, uma análise mais completa. A recomendação da LLM é padrão e bem simples.

**LLM**

Fase 2: Determinar o Formato da Transação

1. Discretização: Converta variáveis numéricas contínuas (como velocidade e tamanho do veículo) em categorias. Por exemplo:
  * Velocidade: Baixa (0-40 km/h), Média (41-60 km/h), Alta (61+ km/h)
  * Tamanho: Pequeno (moto), Médio (carro), Grande (ônibus/caminhão)
  * Horário: Madrugada, Manhã, Tarde, Noite, etc.

2. Criação de Transações: Defina o que constitui uma "transação" para a mineração de padrões. Algumas opções:

  1. Por Registro: Cada linha da tabela é uma transação, contendo as características do veículo e o horário.

  2. Por Intervalo de Tempo: Agrupe os dados em intervalos de tempo (15 minutos, 30 minutos, 1 hora) e crie transações com as características dos veículos que passaram naquele intervalo.

**Crítica**

A LLM não gerou o código necessário para converter as variáveis contínuas, porém gerou uma boa forma de se segmentar as variáveis contínuas, com relação à velocidade mais uma categoria, Muit Alta (100+ Km/h), seria interessante. Com relação ao horário a definição do horário de pico da manhã e tarde seriam também interessantes.

**LLM**

Fase 3: Processamento (Mineração de Padrões)

  1. Preparação dos Dados: Converta os dados para o formato exigido pelo algoritmo de mineração de padrões escolhido (Apriori, FP-Growth, Eclat, etc.). A biblioteca mlxtend em Python oferece implementações desses algoritmos.

  2. Execução do Algoritmo: Execute o algoritmo escolhido com os parâmetros apropriados (suporte mínimo, confiança mínima, etc.).

In [ ]:
# Supondo que 'df_transacoes' é o seu DataFrame de transações
frequent_itemsets = apriori(df_transacoes, min_support=0.05, use_colnames=True)
print(frequent_itemsets)


**Crítica**

A bibliteca não possui a implementação do algoritmo Eclat ele não trata das descobertas de regras, medidas de utilidade e não trata e não faz uma análise dos dados de forma sequencial. As recomendações são muito vagas e não refletem uma análise e programação mais profunda. Poderiam ser informados mais opções de bibliotecas.

**LLM**

Fase 4: Reporte

1. Análise dos Resultados: Interprete os padrões frequentes encontrados. Quais combinações de características ocorrem com frequência? Existem padrões interessantes relacionados a horários, tipos de veículos e velocidade?

2. Visualização: Crie gráficos e tabelas para visualizar os resultados e facilitar a compreensão.

3. Relatório Final: Escreva um relatório detalhando o processo, os resultados e as conclusões. Inclua:

  1. Descrição do problema e dos dados
  2. Metodologia utilizada (pré-processamento, mineração de padrões)
  3. Resultados e interpretação
  4. Limitações do estudo
  5. Sugestões para trabalhos futuros

Ferramentas e Tecnologias:

 * Linguagem de Programação: Python

 * Bibliotecas Python: pandas, mlxtend, matplotlib (para visualização), seaborn (para visualização), json, glob

 * Banco de Dados (opcional): PostgreSQL, MySQL, SQLite

 * Algoritmos de Mineração de Padrões: Apriori, FP-Growth, Eclat (implementados na mlxtend)

 * Ambiente de Desenvolvimento: Jupyter Notebook, VS Code, PyCharm

Lembre-se de documentar cada etapa do processo e justificar as escolhas feitas. Isso tornará seu trabalho mais robusto e replicável. Comece com um conjunto de dados menor para testar e refinar seu código antes de processar todo o conjunto de dados.

**Crítica**

O processo final de análise dos resultados é bem complexo, e a LLM o resumiu de forma muito simples em forma de tópicos.

Em seguida o seguinte prompot foi enviado: Iremos utilizar um agrupamento por tempo, como seria melhor e como deve interpretar os resultados?

**LLM**

Utilizar o agrupamento por tempo adiciona uma dimensão interessante à análise, permitindo identificar padrões que variam ao longo do dia. Aqui está como você pode fazer isso e interpretar os resultados:

1. Definindo o Intervalo de Tempo:

  Escolha um intervalo de tempo apropriado para o agrupamento. As opções comuns incluem:

  * 15 minutos: Oferece granularidade fina, mas pode resultar em muitos grupos e padrões complexos.

  * 30 minutos: Um bom equilíbrio entre granularidade e complexidade.

  * 1 hora: Menos granular, mas mais fácil de interpretar e pode revelar padrões diários mais amplos.

 * Por período do dia: Manhã, tarde, noite, madrugada. Útil para identificar padrões gerais em cada período.

A escolha do intervalo depende do seu objetivo de análise e da natureza dos dados. Comece com um intervalo maior (1 hora) e, se necessário, refine para intervalos menores.

2. Agrupando os Dados com Pandas:

 * Use a função groupby() do pandas para agrupar os dados pelo intervalo de tempo escolhido. Primeiro, certifique-se de que a coluna DATA_HORA esteja no formato datetime.

In [ ]:
import pandas as pd

# Converte a coluna 'DATA_HORA' para datetime
df['DATA_HORA'] = pd.to_datetime(df['DATA_HORA'])

# Agrupa por hora
df_agrupado = df.groupby(pd.Grouper(key='DATA_HORA', freq='H'))

# Exemplo: calcular a velocidade média por hora
velocidade_media_por_hora = df_agrupado['VELOCIDADE_AFERIDA'].mean()
print(velocidade_media_por_hora)

# Criando transações por hora com as características dos veículos
transacoes = []
for nome_grupo, grupo in df_agrupado:
    transacao = []
    for index, row in grupo.iterrows():
        transacao.append(f"Velocidade_{row['VELOCIDADE_AFERIDA_DISCRETIZADA']}")  # Usando a velocidade discretizada
        transacao.append(f"Classificacao_{row['CLASSIFICACAO']}")
        # Adicione outras características relevantes
    transacoes.append(transacao)

# Agora 'transacoes' é uma lista de listas, onde cada lista interna representa uma hora e contém as características dos veículos que passaram naquela hora.


**Crítica**
A LLM não entedeu que iremos particionar as transações em unidades sequenciais para realizar a mineração de padrões sequencias. E não realizou a mineração de sequencias frequentes,

**LLM**

3. Mineração de Padrões:

Aplique o algoritmo de mineração de padrões escolhido às transações geradas no passo anterior. Lembre-se de que agora cada transação representa um intervalo de tempo.

4. Interpretando os Resultados:

Os padrões frequentes encontrados representarão combinações de características que ocorrem com frequência em determinados intervalos de tempo. Por exemplo:

"Velocidade_Alta" e "Classificacao_Moto" às 22h: Indica que motos com alta velocidade são comuns às 22h.

"Velocidade_Baixa" e "Classificacao_Caminhao" das 7h às 8h: Sugere congestionamento de caminhões no horário de pico da manhã.

"Velocidade_Media" e "Faixa_Esquerda" das 18h às 19h: Indica que a faixa da esquerda é usada principalmente por veículos com velocidade média durante o horário de pico da tarde.

Visualização:

Gráficos de Linha: Mostre a frequência de certos padrões ao longo do tempo.

Heatmaps: Visualize a frequência de padrões em diferentes horários do dia e dias da semana.

Considerações Adicionais:

Tendências Temporais: Analise como os padrões mudam ao longo do tempo. Existem padrões sazonais? Os padrões mudam nos finais de semana?

Comparação entre Vias: Compare os padrões encontrados em diferentes vias para identificar diferenças no comportamento do tráfego.

Contexto Externo: Considere fatores externos que podem influenciar os padrões, como eventos, feriados, condições climáticas, etc.

Ao agrupar por tempo e analisar os padrões frequentes, você poderá obter insights valiosos sobre o comportamento do tráfego em diferentes horários e identificar áreas que precisam de atenção especial em termos de segurança e fluxo de veículos. Lembre-se de documentar suas descobertas e conclusões de forma clara e concisa.

**Crítica**

A LLM recomendou análises interessantes como padrões sazonais, eventos, feriados, condições climáticas, etc. O que enriqueceria muito o trabalho. Porém ela foi vaga e muito centrada em tópicos.